In [1]:
import os
import sys
import time
import pennylane as qml
from pennylane import numpy as np

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
from measurement_memory import *
from pauli_grouping import *
from Experiment_Molecular import measurement_rotation, single_qubit_basis_rotation,\
                                 givens_rotation, phase_rotation
from Hamiltonians.utils import read_molecule_Hamiltonian

In [35]:
def To_pennylane(Hg:list):
    obs = []
    coeffs = []
    for i in range(len(Hg)):
        for j,p in enumerate(str_to_Pauli(Hg[i][0])):
            obs.append(p)
            coeffs.append(Hg[i][1][j])
    qmlH = qml.Hamiltonian(coeffs, obs)
    return qmlH

## H2

In [3]:
# Define ansatz
N = 4
params = 2*np.pi*np.random.rand(N*2)
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

#### 1. Original H

In [47]:
H = read_molecule_Hamiltonian(mol="H2", grouping_type="GC", directory='.')
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_raw(x):
    ansatz(x)
    return qml.expval(qmlH)

circuit_raw(params)

array(-0.19336225)

#### 2. QWC evaluation

In [51]:
dev = qml.device("lightning.qubit", wires=N, shots=99999)
@qml.qnode(dev)
def circuit_qwc(x, M):
    ansatz(x)
    single_qubit_basis_rotation(M)
    return qml.counts()

def evaluate(x):
    E = 0
    for h in H:
        M = is_QWC(h[0], return_basis=True)
        E += evaluate_eigenstate(circuit_qwc(params, M), h)
    return E

evaluate(params)

-0.19345781038267046

#### 3. GC evaluation

In [54]:
H = read_molecule_Hamiltonian(mol="H2", grouping_type="GC", directory='.')
Hg, basis = basis_transformation(H)

dev = qml.device("lightning.qubit", wires=N, shots=99999)
@qml.qnode(dev)
def circuit_gc(x, T, Q):
    ansatz(x)
    measurement_rotation(grouping_type="GC", T=T, Q=Q)
    return qml.counts()

def evaluate(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        E += evaluate_eigenstate(circuit_gc(x, T, Q), h)
    return E

evaluate(params)

-0.19468929994016992

#### 4. FG evaluation

In [4]:
# Reference H from OpenFermion
H = read_molecule_Hamiltonian(mol="H2", grouping_type="ref", directory='.')
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)

ref = circuit_fg(params)
print(ref)

-0.12180387151674325


In [5]:
H, G, P = read_molecule_Hamiltonian(mol="H2", grouping_type="FG", directory='.')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, g, p, qmlH):
    ansatz(x)
    measurement_rotation(grouping_type="FG", G=g, P=p)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, g, p, qmlH)
    return E

fge = evaluate(params)
print(fge)

-0.12180387151674332


In [6]:
(fge-ref)/ref

5.696776151284652e-16

# H4

In [15]:
# Define ansatz
N = 8
p = 3
params = 2*np.pi*np.random.rand(N*(p+1))
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    for j in range(p):
        for i in range(N-1):
            qml.CNOT(wires=[i,i+1])
        for i in range(N):
            qml.RX(x[i+N], wires=i)

#### 1. Original H

In [85]:
H = read_molecule_Hamiltonian(mol="H4", grouping_type="GC", directory='.')
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_raw(x):
    ansatz(x)
    return qml.expval(qmlH)

circuit_raw(params)

array(3.27325911)

#### 2. QWC evaluation

In [68]:
H = read_molecule_Hamiltonian(mol="H4", grouping_type="QWC", directory='.')

dev = qml.device("lightning.qubit", wires=N, shots=99999)
@qml.qnode(dev)
def circuit_qwc(x, M):
    ansatz(x)
    single_qubit_basis_rotation(M)
    return qml.counts()

def evaluate(x):
    E = 0
    for h in H:
        M = is_QWC(h[0], return_basis=True)
        #print(M)
        E += evaluate_eigenstate(circuit_qwc(x, M), h)
    return E

evaluate(params)

3.0244229597232475

#### 3. GC evaluation

In [60]:
H = read_molecule_Hamiltonian(mol="H4", grouping_type="GC", directory='.')
Hg, basis = basis_transformation(H)

dev = qml.device("lightning.qubit", wires=N, shots=99999)
@qml.qnode(dev)
def circuit_gc(x, T, Q):
    ansatz(x)
    measurement_rotation(grouping_type="GC", T=T, Q=Q)
    return qml.counts()

def evaluate(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        E += evaluate_eigenstate(circuit_gc(x, T, Q), h)
    return E

evaluate(params)

3.0810496319919682

#### 4. FG evaluation

In [16]:
# Reference H from OpenFermion
H = read_molecule_Hamiltonian(mol="H4", grouping_type="ref", directory='.')

qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)
ref = circuit_fg(params)
print(ref)

0.6564768177777335


In [17]:
H, G, P = read_molecule_Hamiltonian(mol="H4", grouping_type="FG", directory='.')

dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    measurement_rotation(grouping_type="FG", G=g, P=p)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

fge = evaluate(params)
print(fge)

0.6614663474515723


In [18]:
(fge-ref)/ref

0.007600465909411808

## LiH

In [23]:
# Define ansatz
N = 12
p = 3
params = 2*np.pi*np.random.rand(N*(p+1))
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    for j in range(p):
        for i in range(N-1):
            qml.CNOT(wires=[i,i+1])
        for i in range(N):
            qml.RX(x[i+N], wires=i)

In [24]:
# Reference H from OpenFermion
H = read_molecule_Hamiltonian(mol="LiH", grouping_type="ref", directory='.')
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)

ref = circuit_fg(params)
print(ref)

-4.225731595698055


In [25]:
H, G, P = read_molecule_Hamiltonian(mol="LiH", grouping_type="FG", directory='.')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    measurement_rotation(grouping_type="FG", G=g, P=p)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

fge = evaluate(params)
print(fge)

-4.225368614836516


In [26]:
(fge-ref)/ref

-8.589775600233541e-05

## H2O

In [31]:
# Define ansatz
N = 14
p = 3
params = 2*np.pi*np.random.rand(N*(p+1))
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    for j in range(p):
        for i in range(N-1):
            qml.CNOT(wires=[i,i+1])
        for i in range(N):
            qml.RX(x[i+N], wires=i)

In [32]:
# Reference H from OpenFermion
H = read_molecule_Hamiltonian(mol="H2O", grouping_type="ref", directory='.')
qmlH = To_pennylane(H)
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x):
    ansatz(x)
    return qml.expval(qmlH)

ref = circuit_fg(params)
print(ref)

-40.349811437003254


In [33]:
H, G, P = read_molecule_Hamiltonian(mol="H2O", grouping_type="FG", directory='.')

dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev)
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    measurement_rotation(grouping_type="FG", G=g, P=p)
    return qml.expval(qmlH)

def evaluate(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        #print(qmlH)
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

fge = evaluate(params)
print(fge)

-40.34729290274728


In [34]:
(fge-ref)/ref

-6.2417497536728e-05